In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install python-dotenv
# import kagglehub
# import os
# from dotenv import load_dotenv

# # Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
# dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# # Sprawdź, czy plik istnieje przed próbą jego wczytania
# if os.path.exists(dotenv_path):
#     load_dotenv(dotenv_path)

#     # Pobierz zmienne środowiskowe
#     kaggle_login = os.getenv('KAGGLE_LOGIN')
#     kaggle_key = os.getenv('KAGGLE_KEY')

#     # # Możesz teraz użyć kaggle_login i kaggle_key
#     # print(f"KAGGLE_LOGIN: {kaggle_login}")
#     # print(f"KAGGLE_KEY: {kaggle_key}")
# else:
#     print(f"Plik .env nie został znaleziony w {dotenv_path}")
# os.environ['KAGGLE_USERNAME'] = kaggle_login
# os.environ['KAGGLE_KEY'] = kaggle_key

# kagglehub.login()


In [ ]:
# print('Importing data sources...')

# competition_path = kagglehub.competition_download('digit-recognizer')

# print('Data source import complete.')

In [ ]:
# # List files in the downloaded directory
# print(f"Files in the downloaded directory ({competition_path}):")
# for dirname, _, filenames in os.walk(competition_path):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Wczytanie danych
# ds_train = pd.read_csv(os.path.join(competition_path, "train.csv"))
# ds_test = pd.read_csv(os.path.join(competition_path, "test.csv"))
ds_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
ds_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(f'Train shape: {ds_train.shape}')
print(f'Test shape: {ds_test.shape}')

In [ ]:
from sklearn.model_selection import train_test_split

X_train_full = ds_train.drop('label', axis=1)
y_train_full = ds_train['label']
y_train = y_train_full

# Normalizacja pikseli

X_train = X_train_full / 255
X_test = ds_test / 255

# # Podział na zbiór treningowy i walidacyjny
# X_train, X_valid = X_train_full[:-2000], X_train_full[-2000:]
# y_train, y_valid = y_train_full[:-2000], y_train_full[-2000:]

In [ ]:
# Ustaw ziarno losowości
import random

SEED = 42
# os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Dodanie wymiaru kanału dla "Splotowej Sieci Neuronowej"
X_train = X_train.values.reshape(-1, 28, 28, 1)
# X_valid = X_valid.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)


In [ ]:
y_train = y_train.astype('int')

In [ ]:
X=X_train
y=y_train

In [ ]:
# Model
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.BatchNormalization(),  # dodaj batch norm
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# === 3. Konfiguracja augmentacji ===
# Tylko dla danych treningowych — nie dla walidacyjnych!
datagen = ImageDataGenerator(
    rotation_range=10,        # losowy obrót w zakresie [-10°, 10°]
    zoom_range=0.1,           # przybliżenie losowo w zakresie [0.9, 1.1]
    width_shift_range=0.1,    # przesunięcie poziome (maks. 10%)
    height_shift_range=0.1    # przesunięcie pionowe (maks. 10%)
)

**Stratified Cross Validation**


In [ ]:
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []
models_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold + 1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = build_model()
    
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,             # zatrzymaj po 5 epokach bez poprawy
        restore_best_weights=True
    )

    datagen.fit(X_train)

    #Trening
    model.fit(
        datagen.flow(X_train, y_train, batch_size=64),  # dane augmentowane w locie
        validation_data=(X_val, y_val),                 # bez augmentacji
        epochs=30,
        callbacks=[early_stop],
        verbose=1
    )

    models_list.append(model)  # zapisujemy model z tego folda
    # Ocena
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"✅ Fold {fold+1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

# Średnia dokładność
mean_acc = np.mean(fold_accuracies)
print(f"\n📊 Średnia dokładność z {skf.n_splits} foldów: {mean_acc:.4f}")

In [ ]:
# === 6. Ensemble predykcji ===
print("\n📊 Ensemble predykcji z 5 modeli...")
all_preds = np.zeros((X_test.shape[0], 10))

for model in models_list:
    pred = model.predict(X_test, verbose=0)
    all_preds += pred

ensemble_preds = np.argmax(all_preds / len(models_list), axis=1)

In [ ]:
# history = model.fit(X_train, y_train,
#                     batch_size=64,
#                     epochs=30,
#                     validation_data=(X_valid, y_valid),
#                     shuffle=False,
#                     callbacks=[early_stop])


# def plot_history(history):
#     plt.figure(figsize=(12, 4))

#     # Accuracy
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['accuracy'], label='train acc')
#     if 'val_accuracy' in history.history:
#         plt.plot(history.history['val_accuracy'], label='val acc')
#     plt.title('Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()

#     # Loss
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['loss'], label='train loss')
#     if 'val_loss' in history.history:
#         plt.plot(history.history['val_loss'], label='val loss')
#     else:
#         print("⚠️ Nie znaleziono 'val_loss' — czy podałeś validation_data?")
#     plt.title('Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# plot_history(history)

In [ ]:
# y_pred_proba = model.predict(X_test)
# y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(ensemble_preds) + 1),
    "Label": ensemble_preds
})

submission.to_csv("07_submission.csv", index=False)